In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image
from scipy.ndimage import label

tree_cond = {"EMPTY": 0, "TREE": 1, "BURNING": 2, "BURNED": 3}
tree_color = {0: "white", 1: "green", 2: "red", 3: "grey", 4: "darkgreen"}
cmap = ListedColormap(tree_color.values())

def forest_grid(size, p_tree):
    return np.random.choice([tree_cond["EMPTY"], tree_cond["TREE"]],
                            size=(size, size), p=[1-p_tree, p_tree])

def get_neighbors(x, y, size, neighborhood):
    if neighborhood == "von_neumann":
        neighbors = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
    elif neighborhood == "moore":
        neighbors = [(x+i, y+j) for i in [-1, 0, 1]
                                for j in [-1, 0, 1] if not (i == 0 and j == 0)]
    else:
        return []
    return [(nx, ny) for nx, ny in neighbors if 0 <= nx < size and 0 <= ny < size]

def find_largest_cluster(forest, neighborhood, highlight=False):
    structure = np.array([[0,1,0],[1,1,1],[0,1,0]]) if neighborhood == "von_neumann" else np.ones((3,3), int)
    tree_mask = (forest == tree_cond["TREE"])
    labeled, num_features = label(tree_mask, structure=structure)

    largest_size = 0
    largest_cluster = 0
    for i in range(1, num_features+1):
        size = np.sum(labeled == i)
        if size > largest_size:
            largest_size = size
            largest_cluster = i

    coords = np.argwhere(labeled == largest_cluster)

    if highlight:
        for x, y in coords:
            forest[x, y] = 4  # ciemnozielony

    return coords

def draw_burning_time(burning_time_options):
    values, probs = zip(*burning_time_options)
    return np.random.choice(values, p=probs)

def spread_fire(forest, size, p_ignite, neighborhood, burning_time, burning_time_options):
    new_forest = forest.copy()
    new_burning_time = burning_time.copy()

    for x in range(size):
        for y in range(size):
            if forest[x, y] == tree_cond["BURNING"]:
                if burning_time[x, y] > 1:
                    new_burning_time[x, y] -= 1
                else:
                    new_forest[x, y] = tree_cond["BURNED"]
                    new_burning_time[x, y] = 0

                for nx, ny in get_neighbors(x, y, size, neighborhood):
                    if forest[nx, ny] in [tree_cond["TREE"], 4]:
                        if np.random.rand() < p_ignite:
                            new_forest[nx, ny] = tree_cond["BURNING"]
                            new_burning_time[nx, ny] = draw_burning_time(burning_time_options)

    return new_forest, new_burning_time

def start_fire(forest_before_fire, burning_time, burning_time_options):
    forest = forest_before_fire.copy()
    for y in range(forest.shape[1]):
        if forest[0, y] == tree_cond["TREE"]:
            forest[0, y] = tree_cond["BURNING"]
            burning_time[0, y] = draw_burning_time(burning_time_options)
    return forest, burning_time

def fire_simulation(size, p_ignite, p_tree, neighborhood, gif_name,
                    M_frames=100, start_method='cluster',
                    burning_time_options=[(1, 0.5), (2, 0.3), (3, 0.2)]):

    forest = forest_grid(size, p_tree)
    burning_time = np.zeros((size, size), dtype=int)
    frames = []

    if start_method == 'cluster':
        cluster_coords = find_largest_cluster(forest, neighborhood, highlight=True)
    else:
        cluster_coords = None

    for step in range(M_frames):
        if step == 0:
            title = f'Początkowy stan'
        elif step == 1:
            if start_method == 'top':
                forest, burning_time = start_fire(forest, burning_time, burning_time_options)
                title = f'Zapalono drzewa w górnym rzędzie'
            elif start_method == 'cluster':
                if cluster_coords is not None and cluster_coords.size > 0:
                    x, y = cluster_coords[np.random.randint(len(cluster_coords))]
                    forest[x, y] = tree_cond["BURNING"]
                    burning_time[x, y] = draw_burning_time(burning_time_options)
                title = f'Zapalono drzewo w największym klastrze'
        else:
            forest, burning_time = spread_fire(forest, size, p_ignite, neighborhood, burning_time, burning_time_options)
            title = f'Symulacja: krok {step}'

        fig, ax = plt.subplots(figsize=(7, 7), dpi=80)
        ax.imshow(forest, cmap=cmap, vmin=0, vmax=4)
        ax.axis('off')
        ax.set_title(title)
        plt.draw()
        fig.canvas.draw()
        buf = fig.canvas.buffer_rgba()
        image = Image.frombytes('RGBA', fig.canvas.get_width_height(), buf)
        frames.append(image)
        plt.close(fig)

    frames[0].save(f'{gif_name}.gif', save_all=True, append_images=frames[1:], loop=0, duration=200)
    print(f"Animacja została zapisana jako '{gif_name}.gif'")


In [5]:
fire_simulation(
    size=50,
    p_ignite=0.7,
    p_tree=0.45,
    neighborhood="moore",
    gif_name="moore_top",
    M_frames=150,
    start_method="top",
    burning_time_options=[(2, 0.3), (5, 0.7)]
)


Animacja została zapisana jako 'moore_top.gif'


In [11]:
fire_simulation(
    size=50,
    p_ignite=1,
    p_tree=0.3,
    neighborhood="von_neumann",
    gif_name="von_neumann_cluster",
    M_frames=150,
    start_method="cluster",
    burning_time_options=[(1, 1)]
)


Animacja została zapisana jako 'von_neumann_cluster.gif'
